In [54]:
from sage.all import *
from pwn import *
import json

import hashlib
from Crypto.Util.number import bytes_to_long, long_to_bytes, inverse
from ecdsa.ecdsa import Public_key, Private_key, Signature, generator_192

from datetime import datetime
from random import randrange

In [55]:
G = generator_192
q = G.order()

packet = {
    "option":"sign_time"
}

host = "socket.cryptohack.org"
port = 13381

remote = remote(host, port)

remote.recvuntil('verify.\n').strip().decode()
request = json.dumps(packet).encode()
remote.sendline(request)

data = remote.recvline(keepends=False).strip().decode()
data = json.loads(data)

print(data)
r = int(data['r'], 16)
s = int(data['s'], 16)

# Có s, r. Vì k được chọn từ 0, 59 nên ta thử từng k để tìm ra d
# Ta có s = k^-1(e + dr) => d = (sk - e)/r
# Dựa vào r = kG. Ta tính lại k
e = bytes_to_long(hashlib.sha1(data['msg'].encode()).digest()) % q
k = 0
for i in range(1, 60, 1):
    R = i * G
    xR = R.x()
    if xR % q == r:
        k = i
        break

inv_r = inverse(r, q)
d = (s * k - e) * inv_r % q
print(d)

# Đã có private key thì ta sẽ tính toán message:
e = bytes_to_long(hashlib.sha1("unlock".encode()).digest()) % q
new_r = 0
while new_r == 0:
    k = randint(1, q-1)
    new_r = (k * G).x()

s = 0
while s == 0:
    inv_k = inverse(k, q)
    s = inv_k * (e  + d * new_r) % q


packet = {
    "option": "verify",
    'msg': "unlock",
    'r': str(hex(new_r)),
    's': str(hex(s))
}
request = json.dumps(packet).encode()
remote.sendline(request)

data = remote.recvline(keepends=False).strip().decode()

data = json.loads(data)
print(data['flag'])
remote.close()



[x] Opening connection to socket.cryptohack.org on port 13381
[x] Opening connection to socket.cryptohack.org on port 13381: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13381: Done


/tmp/ipykernel_1708/1065291550.py:13: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  remote.recvuntil('verify.\n').strip().decode()


{'msg': 'Current time is 11:33', 'r': '0x818a4d308b1cabb74e9e8f2ba8d27c9e1d9d375ab980388f', 's': '0x38619a5401164b5e9c713b2cf2bff8df46b9b18bbc0b3acc'}
3425929618800532753808649405485497985831912025742212253314
crypto{ECDSA_700_345y_70_5cr3wup}
[*] Closed connection to socket.cryptohack.org port 13381
